# Benchmark GPU vs CPU & multi-host vs single host
---

This notebook can be used to benchmark performance using CPU, a single GPU or many GPUs.

<font color='red'> <h3>Tested with TensorFlow 2.4.0</h3></font>

<p>
<h1>Machine Learning on <a href="https://github.com/logicalclocks/hopsworks">Hopsworks
</a></h1> 
</p>

![hops.png](../../../images/hops.png)

## The `hops` python module

`hops` is a helper library for Hops that facilitates development by hiding the complexity of running applications and iteracting with services.

Have a feature request or encountered an issue? Please let us know on <a href="https://github.com/logicalclocks/hops-util-py">github</a>.

### Using the `experiment` module

To be able to run your Machine Learning code in Hopsworks, the code for the whole program needs to be provided and put inside a wrapper function. Everything, from importing libraries to reading data and defining the model and running the program needs to be put inside a wrapper function.

The `experiment` module provides an api to Python programs such as TensorFlow, Keras and PyTorch on a Hopsworks on any number of machines and GPUs.

An Experiment could be a single Python program, which we refer to as an **Experiment**. 

Grid search or genetic hyperparameter optimization such as differential evolution which runs several Experiments in parallel, which we refer to as **Parallel Experiment**. 

ParameterServerStrategy, CollectiveAllReduceStrategy and MultiworkerMirroredStrategy making multi-machine/multi-gpu training as simple as invoking a function for orchestration. This mode is referred to as **Distributed Training**.

### Using the `tensorboard` module
The `tensorboard` module allow us to get the log directory for summaries and checkpoints to be written to the TensorBoard we will see in a bit. The only function that we currently need to call is `tensorboard.logdir()`, which returns the path to the TensorBoard log directory. Furthermore, the content of this directory will be put in as a Dataset in your project's Experiments folder.

The directory could in practice be used to store other data that should be accessible after the experiment is finished.
```python
# Use this module to get the TensorBoard logdir
from hops import tensorboard
tensorboard_logdir = tensorboard.logdir()
```

### Using the `hdfs` module
The `hdfs` module provides a method to get the path in HopsFS where your data is stored, namely by calling `hdfs.project_path()`. The path resolves to the root path for your project, which is the view that you see when you click `Data Sets` in HopsWorks. To point where your actual data resides in the project you to append the full path from there to your Dataset. For example if you create a mnist folder in your Resources Dataset, the path to the mnist data would be `hdfs.project_path() + 'Resources/mnist'`

```python
# Use this module to get the path to your project in HopsFS, then append the path to your Dataset in your project
from hops import hdfs
project_path = hdfs.project_path()
```

```python
# Downloading the mnist dataset to the current working directory
from hops import hdfs
mnist_hdfs_path = hdfs.project_path() + "Resources/mnist"
local_mnist_path = hdfs.copy_to_local(mnist_hdfs_path)
```

### Documentation
See the following links to learn more about running experiments in Hopsworks

- <a href="https://hopsworks.readthedocs.io/en/latest/hopsml/experiment.html">Learn more about experiments</a>
<br>
- <a href="https://hopsworks.readthedocs.io/en/latest/hopsml/hopsML.html">Building End-To-End pipelines</a>
<br>
- Give us a star, create an issue or a feature request on  <a href="https://github.com/logicalclocks/hopsworks">Hopsworks github</a>

### Managing experiments
Experiments service provides a unified view of all the experiments run using the `experiment` module.
<br>
As demonstrated in the gif it provides general information about the experiment and the resulting metric. Experiments can be visualized meanwhile or after training in a TensorBoard.
<br>
<br>
![Image7-Monitor.png](../../../images/experiments.gif)

In [5]:
def wrapper():
    import tensorflow as tf
    
    # Wrapper for keras_applications, you can import any model you want to try (like ResNet50)
    from tensorflow.keras.applications import ResNet50

    import numpy as np
    
    from hops import tensorboard
    
    # Utility module for getting number of GPUs accessible by the container (Spark Executor)
    from hops import devices

    batch_size = 8 # Number of samples to process on each GPU every iteration
    
    # Image dimensions
    height = 224
    width = 224
    channels = 3
    num_classes = 1000
    
    num_iterations = 5000 # Number of iterations, increase to run longer
    

    log_dir = tensorboard.logdir()
    
    # Read synthetic data (can be replaced with real data)
    def input_fn():
      data = np.random.random((batch_size, height, width, channels)).astype(np.float32)
      labels = np.random.random((batch_size, num_classes))
      dataset = tf.data.Dataset.from_tensor_slices((data, labels))
      dataset = dataset.repeat(num_iterations)
      dataset = dataset.batch(batch_size)
      return dataset    
    
    tf.keras.backend.set_learning_phase(True)

    
    # Define distribution strategy
    strategy = tf.distribute.MirroredStrategy()


    with strategy.scope():
        model = ResNet50(weights=None, input_shape=(height, width, channels), classes=num_classes)

        optimizer = tf.keras.optimizers.RMSprop(0.2)
        model.compile(loss='categorical_crossentropy', optimizer=optimizer)


    callbacks = [
        tf.keras.callbacks.TensorBoard(log_dir=log_dir),
        tf.keras.callbacks.ModelCheckpoint(filepath=log_dir),
    ]
    model.fit(input_fn(), 
        verbose=0,
        epochs=3, 
        steps_per_epoch=5,
        validation_data=input_fn(),
        callbacks=callbacks
    )
    model.evaluate(input_fn())
        

In [6]:
from hops import experiment
experiment.launch(wrapper, local_logdir=True)

Finished Experiment 

('hdfs://rpc.namenode.service.consul:8020/Projects/demo/Experiments/application_1594231828166_0163_3', {'metric': None, 'log': 'Experiments/application_1594231828166_0163_3/output.log'})